In [1]:
import fml_lib
import numpy as np
import pandas as pd
import scipy.stats as ss
import os.path
from math import sqrt, exp, log
import matplotlib.pyplot as plt
from mp_engine import mpPandasObj

In [2]:
dollar_bars = pd.read_csv("USA500IDXUSD_bars.csv", parse_dates=[['Date', 'Timestamp']])
dollar_bars = dollar_bars.set_index("Date_Timestamp")
close = dollar_bars['Close']
daily_volatility = fml_lib.getDailyVol(dollar_bars.Close,span0=100)


C:\Users\dstek\AppData\Local\Temp\ipykernel_9564\1907931566.py:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  dollar_bars = pd.read_csv("USA500IDXUSD_bars.csv", parse_dates=[['Date', 'Timestamp']])


In [3]:

if os.path.exists("./t_events.csv"):
    t_events = pd.read_csv("t_events.csv", index_col=[0])
    t_events = t_events.index
else:
    t_events = fml_lib.getTEvents(dollar_bars.Close, daily_volatility)
    t_events_df = pd.DataFrame(index= t_events)
    t_events_df.to_csv("./t_events_csv", index=True)

In [4]:
t_events = pd.to_datetime(t_events)
print(t_events)

DatetimeIndex(['2023-07-04 22:00:10', '2023-07-04 22:00:40',
               '2023-07-04 22:00:50', '2023-07-04 22:01:10',
               '2023-07-04 22:01:30', '2023-07-04 22:01:50',
               '2023-07-04 22:02:10', '2023-07-04 22:02:20',
               '2023-07-04 22:02:40', '2023-07-04 22:03:00',
               ...
               '2024-01-25 03:52:20', '2024-01-25 03:53:40',
               '2024-01-25 03:54:50', '2024-01-25 03:55:10',
               '2024-01-25 03:55:40', '2024-01-25 03:56:50',
               '2024-01-25 03:57:00', '2024-01-25 03:57:20',
               '2024-01-25 03:58:30', '2024-01-25 03:58:40'],
              dtype='datetime64[ns]', length=814094, freq=None)


In [5]:


#adds the vertical barrier num_days out from the time of the event
numDays = 10
t1=close.index.searchsorted(t_events+pd.Timedelta(days=numDays))
t1=t1[t1<close.shape[0]]
t1=pd.Series(close.index[t1],index=t_events[:t1.shape[0]]) # NaNs at end
print(t1)

2023-07-04 22:00:10   2023-07-16 22:00:00
2023-07-04 22:00:40   2023-07-16 22:00:00
2023-07-04 22:00:50   2023-07-16 22:00:00
2023-07-04 22:01:10   2023-07-16 22:00:00
2023-07-04 22:01:30   2023-07-16 22:00:00
                              ...        
2024-01-15 03:54:40   2024-01-25 03:54:50
2024-01-15 03:56:00   2024-01-25 03:56:50
2024-01-15 03:57:00   2024-01-25 03:57:00
2024-01-15 03:57:30   2024-01-25 03:58:30
2024-01-15 03:58:40   2024-01-25 03:58:40
Name: Date_Timestamp, Length: 769572, dtype: datetime64[ns]


In [6]:
min_ret= 0.005
trgt = pd.Series(0.01, index=t_events)
if os.path.exists("./events.csv"):
    events = pd.read_csv("events.csv", index_col=[0])
    events = events.dropna()
else:
    events = fml_lib.getEvents(close, t_events, 0.02,trgt ,min_ret,t1)
    events.to_csv("events.csv")
print(events)

                                      t1  trgt
2023-07-04 22:00:10  2023-07-04 22:17:50  0.01
2023-07-04 22:00:40  2023-07-04 22:17:50  0.01
2023-07-04 22:00:50  2023-07-04 22:15:30  0.01
2023-07-04 22:01:10  2023-07-04 22:17:50  0.01
2023-07-04 22:01:30  2023-07-04 22:17:50  0.01
...                                  ...   ...
2024-01-25 03:50:30  2024-01-25 03:56:50  0.01
2024-01-25 03:51:30  2024-01-25 03:54:50  0.01
2024-01-25 03:51:40  2024-01-25 03:56:50  0.01
2024-01-25 03:51:50  2024-01-25 03:54:50  0.01
2024-01-25 03:52:00  2024-01-25 03:56:50  0.01

[814081 rows x 2 columns]


In [7]:

out = fml_lib.getBins(events, close, numDays)

[ True  True  True ...  True  True  True]


In [8]:
print(out[out['bin'] == 1])
print(out)

                          ret  bin
2023-07-04 22:00:10  0.000226  1.0
2023-07-04 22:00:40  0.000223  1.0
2023-07-04 22:00:50  0.000220  1.0
2023-07-04 22:01:10  0.000226  1.0
2023-07-04 22:01:30  0.000224  1.0
...                       ...  ...
2024-01-25 03:50:30  0.000207  1.0
2024-01-25 03:51:30  0.000216  1.0
2024-01-25 03:51:40  0.000208  1.0
2024-01-25 03:51:50  0.000215  1.0
2024-01-25 03:52:00  0.000207  1.0

[412467 rows x 2 columns]
                          ret  bin
2023-07-04 22:00:10  0.000226  1.0
2023-07-04 22:00:40  0.000223  1.0
2023-07-04 22:00:50  0.000220  1.0
2023-07-04 22:01:10  0.000226  1.0
2023-07-04 22:01:30  0.000224  1.0
...                       ...  ...
2024-01-25 03:50:30  0.000207  1.0
2024-01-25 03:51:30  0.000216  1.0
2024-01-25 03:51:40  0.000208  1.0
2024-01-25 03:51:50  0.000215  1.0
2024-01-25 03:52:00  0.000207  1.0

[814081 rows x 2 columns]


Want to find sample weights for use in the boosting fit

In [9]:
print(events)

                                      t1  trgt
2023-07-04 22:00:10  2023-07-04 22:17:50  0.01
2023-07-04 22:00:40  2023-07-04 22:17:50  0.01
2023-07-04 22:00:50  2023-07-04 22:15:30  0.01
2023-07-04 22:01:10  2023-07-04 22:17:50  0.01
2023-07-04 22:01:30  2023-07-04 22:17:50  0.01
...                                  ...   ...
2024-01-25 03:50:30  2024-01-25 03:56:50  0.01
2024-01-25 03:51:30  2024-01-25 03:54:50  0.01
2024-01-25 03:51:40  2024-01-25 03:56:50  0.01
2024-01-25 03:51:50  2024-01-25 03:54:50  0.01
2024-01-25 03:52:00  2024-01-25 03:56:50  0.01

[814081 rows x 2 columns]


In [10]:
temp = list(set(pd.to_datetime(events.t1.index)) & set(pd.to_datetime(events.index)) )
print(len(temp))
print(len(events))

814081
814081


In [11]:
test_co_events = mpPandasObj(fml_lib.mpNumCoEvents,('molecule',events.index),24, closeIdx=close.index,t1=events['t1'])

if os.path.exists("./t_events.csv"):  
    numCoEvents = pd.read_csv("./co_events.csv", index_col=[0])
else:
    numCoEvents = fml_lib.get_co_events(close,events)
    numCoEvents.to_csv("./co_events.csv", index=True)
print(numCoEvents)
print(test_co_events)

KeyboardInterrupt: 

In [ ]:
ret=np.log(close).diff() # log-returns, so that they are additive
print(ret)

Date_Timestamp
2023-07-02 22:00:00         NaN
2023-07-02 22:00:10    0.000180
2023-07-02 22:00:20   -0.000071
2023-07-02 22:00:30   -0.000110
2023-07-02 22:00:40    0.000110
                         ...   
2024-01-25 03:57:00   -0.000106
2024-01-25 03:57:10    0.000000
2024-01-25 03:57:20    0.000003
2024-01-25 03:58:30   -0.000054
2024-01-25 03:58:40    0.000003
Name: Close, Length: 840953, dtype: float64


In [ ]:
print(numCoEvents.dropna())

                      0
Date_Timestamp         
2023-07-04 22:00:10   1
2023-07-04 22:00:40   2
2023-07-04 22:00:50   3
2023-07-04 22:01:10   4
2023-07-04 22:01:30   5
...                  ..
2024-01-25 03:53:40  42
2024-01-25 03:54:50  38
2024-01-25 03:55:10  17
2024-01-25 03:55:40  17
2024-01-25 03:56:50  17

[837059 rows x 1 columns]


In [ ]:
numThreads = 12
#numCoEvents=numCoEvents.loc[~numCoEvents.index.duplicated(keep='last')]
#numCoEvents=numCoEvents.reindex(close.index).fillna(0)
#print(numCoEvents)
out['tW']=mpPandasObj(fml_lib.mpSampleTW,('molecule',events.index),numThreads, t1=events['t1'],numCoEvents=numCoEvents)
print(out.dropna())

2024-04-04 20:52:29.488374 100.0% mpSampleTW done after 0.09 minutes. Remaining 0.0 minutes.


                          ret  bin        tW
2023-07-04 22:00:10  0.000226  1.0  0.123050
2023-07-04 22:00:40  0.000223  1.0  0.096475


c:\Users\dstek\Documents\ECE479\mp_engine.py:123: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df0=df0._append(i)


In [ ]:

sample_weights = mpPandasObj(fml_lib.mpSampleW,('molecule',events.index),numThreads, t1=events['t1'],numCoEvents=numCoEvents,close=close)
print(sample_weights)

2024-04-04 20:52:37.894340 100.0% mpSampleW done after 0.12 minutes. Remaining 0.0 minutes..


2023-07-04 22:00:10    0.000883
2023-07-04 22:00:40    0.000719
2023-07-04 22:00:50    0.000427
dtype: float64


c:\Users\dstek\Documents\ECE479\mp_engine.py:123: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df0=df0._append(i)
